Folder Path and libraries

In [16]:
import pandas as pd
import re
import os
import pandas as pd
import datetime

#CONST

folder_path = r"D:\data\CGIAR\UNFCCC"

1. Adaptation Fund

Add the project number to the original CSV file downloaded from the 'adaptation_fund_path' website. Also, identify projects listed on the website that have no corresponding documents in 'AF_projects_not_in_documents'. Date files indicate when the data collection. 


In [7]:
import pandas as pd
import datetime

# Paths to your CSV files

adaptation_fund_path = os.path.join(folder_path, "processing_stage_movebackmain", "AF_20240403.csv")  #the file downloaded as csv from ADF
project_documents_path = os.path.join(folder_path, "processing_stage_movebackmain", "AF_project_documents_links_with_titles_unique.csv")



# Read the CSV files
df_adaptation_fund = pd.read_csv(adaptation_fund_path, on_bad_lines='skip',encoding='utf-8')
df_project_documents = pd.read_csv(project_documents_path, on_bad_lines='skip',encoding='utf-8')

# Ensure 'project_link' is in df_project_documents to proceed with the merge
df_joined = pd.merge(df_adaptation_fund, df_project_documents[['project_link', 'project_title', 'document_link', 'project_no']], left_on='projecturl', right_on='project_link', how='left')

    # Remove the 'project_link' column
df_joined.drop('project_link', axis=1, inplace=True)


current_time = datetime.datetime.now().strftime("%Y%m%d")
#find if projects no documents
projects_not_in_documents = df_adaptation_fund[~df_adaptation_fund['projecturl'].isin(df_project_documents['project_link'])]
filename = f"AF_projects_not_in_documents_{current_time}.csv"
projects_not_in_documents.to_csv(rf"C:\Users\david\My Drive\data\analysis_git_data\cgiar\{filename}", index=False)

# drop if no project number

df_joined_notna = df_joined[df_joined['project_no'].notnull()]
#add org column for master file
df_joined_notna['org'] = 'AF'
df_joined_notna.rename(columns={'projecturl': 'url', 'grantamount':'grant_amount', 'projecttitle': 'title', 'ie':'entity', 'fy_approval':'year'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.  

df_joined_notna['theme'] = 'Adaptation'

current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"AF_joined_projects_{current_time}.csv"

full_path = os.path.join(folder_path, filename)

df_joined_notna.to_csv(full_path, index=False,encoding='utf-8')





C:\Users\david\AppData\Local\Temp\ipykernel_10076\992744980.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_notna['org'] = 'AF'
C:\Users\david\AppData\Local\Temp\ipykernel_10076\992744980.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_notna.rename(columns={'projecturl': 'url', 'grantamount':'grant_amount', 'projecttitle': 'title', 'ie':'entity', 'fy_approval':'year'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.
C:\Users\david\AppData\Local\Temp\ipykernel_10076\992744980.py:35: Setting

2. GCF 

In [8]:


gcf_projects = pd.read_csv(os.path.join(folder_path, "processing_stage_movebackmain", "gcf_projects_20240403.csv"),encoding='utf-8') #the file downloaded as csv from GCF
gcf_projects['org'] = 'GCF'
gcf_projects.rename(columns={'Ref #': 'project_no', 'Countries': 'country', 'Project Name': 'title', 'Sector': 'sector', 'Entity': 'entity', 'BM': 'year', 'FA Financing': 'grant_amount', 'Theme': 'theme'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.  


current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"GCF_joined_projects_{current_time}.csv"
file_path = os.path.join(folder_path, filename)

gcf_projects.to_csv(file_path, index=False,encoding='utf-8')


3. GEF

In [9]:
gef_df = pd.read_csv(os.path.join(folder_path, "processing_stage_movebackmain", "gef_projects.csv"),encoding='utf-8') #the file downloaded as csv from GEF with filter climate change
gef_df['org'] = 'GEF'
gef_df.rename(columns={'ID': 'project_no', 'Countries': 'country', 'Title': 'title', 'Focal Areas': 'sector', 'Agencies': 'entity', 'GEF Grant': 'grant_amount', 'Status':'status', 'Approval FY': 'year'}, inplace=True) #rename for consistency with master

base_url = 'https://www.thegef.org/projects-operations/projects/'

gef_df['url'] = gef_df['project_no'].apply(lambda x: base_url + str(x))


gef_df['theme'] = 'NA'



current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"GEF_joined_projects_{current_time}.csv"
file_path = os.path.join(folder_path, filename)
gef_df.to_csv(file_path, index=False,encoding='utf-8')


Creation of the Master file

In [45]:
import pandas as pd
import pycountry
from unidecode import unidecode
import re
import datetime
import os

# Load the CSV file with UTF-8 encoding
data = pd.read_csv("D:\\data\\CGIAR\\UNFCCC\\master_file_af_gcf_gef_20240506.csv", encoding='utf-8')

# Extended mapping for normalizing country names and their ISO3 codes
country_normalization_mapping = {
    'Libyan Arab Jamahiriya': 'Libya',
    'Côte d’Ivoire': 'CIV',
    'Cote d\'Ivoire': 'CIV',
    'C&ocirc;te d\'Ivoire':'CIV',
    'Micronesia, Federated States of': 'FSM',
    'Micronesia (Federated States of)': 'FSM',
    'Micronesia': 'FSM',
    'Comoros (the)': 'Comoros',
    'Democratic Republic of the Congo (the)': 'COD',
    'Lao People\'s Democratic Republic (the)': 'LAO',
    'State of Palestine': 'PSE',
    'Niger (the)': 'NER',
    'Philippines (the)': 'PHL',
    'Bolivia (Plurinational State of)': 'BOL',
    'Lao PDR': 'LAO',
    'Timor Leste': 'TLS',
    'Congo DR': 'COD',
    'Bosnia-Herzegovina': 'BIH',
    'Africa, Regional': 'AFR',
    'St. Kitts And Nevis': 'KNA',
    'St. Lucia': 'LCA',
    'St. Vincent and Grenadines': 'VCT',
    'Korea DPR': 'PRK',
    'Regional, Timor Leste': 'Regional, Timor-Leste',
    'Regional, Bosnia-Herzegovina': 'Regional, Bosnia and Herzegovina',
    'Regional, Micronesia': 'Regional, Micronesia (Federated States of)',
    'Regional, Cote d\'Ivoire': 'Regional, CIV',
    'Regional, St. Lucia': 'Regional, Saint Lucia',
    'Cote d\'Ivoire, Global': 'CIV, Global',
    'Turkiye': 'TUR',
}

# Normalize country names
def normalize_country_name(country_name):
    # Handle prefixes like 'Regional,'
    prefixes = ['Regional,', 'Global,']
    for prefix in prefixes:
        if country_name.startswith(prefix):
            base_name = country_name[len(prefix):].strip()
            normalized_base_name = country_normalization_mapping.get(base_name, base_name)
            return f'{prefix} {normalized_base_name}'
    
    # Handle multiple countries
    if ',' in country_name:
        countries = country_name.split(',')
        normalized_countries = [country_normalization_mapping.get(country.strip(), country.strip()) for country in countries]
        return ', '.join(normalized_countries)
    
    # Single country
    normalized_name = country_normalization_mapping.get(country_name, country_name)
    return unidecode(normalized_name)

# Apply the normalization function to the 'country' column
data['country'] = data['country'].apply(normalize_country_name)

# Function to get ISO3 code
def get_iso3(country_name):
    if country_name in country_normalization_mapping:
        return country_normalization_mapping[country_name]
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Handle multiple country names in a cell
def convert_countries_to_iso3(countries):
    if ',' in countries:
        # Split the countries and convert each one
        country_list = countries.split(',')
        iso3_list = [get_iso3(country.strip()) for country in country_list]
        return ', '.join([code for code in iso3_list if code])
    else:
        # Single country
        return get_iso3(countries)

data['country_iso3'] = data['country'].apply(convert_countries_to_iso3)

# Counting None values in the 'country_iso3' column
none_count = data['country_iso3'].isna().sum()
print("Count of None in 'country_iso3':", none_count)

# Function to extract country names and convert to ISO3
def extract_countries_to_iso3(title):
    # Patterns to extract country names
    pattern = r'\b(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b'
    
    matches = re.findall(pattern, title)
    
    # Filter out words that are not country names
    country_names = []
    for name in matches:
        try:
            country = pycountry.countries.lookup(name)
            country_names.append(country.alpha_3)
        except LookupError:
            continue
    
    # Join the ISO3 codes as a comma-separated string
    return ', '.join(country_names) if country_names else None

filtered_df = data[data['country_iso3'].isna() | data['country_iso3'].eq('')]
# Apply the function to the entire DataFrame
filtered_df['country_iso3'] = filtered_df['title'].apply(extract_countries_to_iso3)

# Merge filtered_df back to the original data DataFrame
data.update(filtered_df[['country_iso3']])
# Counting None values in the 'country_iso3' column
none_count = data['country_iso3'].isna().sum()
print("Count of None in 'country_iso3':", none_count)

current_time = datetime.datetime.now().strftime("%Y%m%d")
data.to_csv("D:\\data\\CGIAR\\UNFCCC\\master.csv", index=False, encoding='utf-8')

#check for the files

af_df = pd.read_csv(os.path.join(folder_path, "AF_joined_projects_20240517.csv"), encoding='utf-8')
gcf_df = pd.read_csv(os.path.join(folder_path, "GCF_joined_projects_20240517.csv"), encoding='utf-8')
gef_df = pd.read_csv(os.path.join(folder_path, "GEF_joined_projects_20240517.csv"), encoding='utf-8')

print(f"Total projects from the Adaptation Fund are {len(af_df)}. On the website, there are 149. Missing or not in dataset: {149 - len(af_df)}.")
print(f"Total projects from the GCF are {len(gcf_df)}. On the website, there are 253. Missing or not in dataset: {253 - len(gcf_df)}.")
print(f"Total projects from the GEF are {len(gef_df)}. On the website, there are 2402. Missing or not in dataset: {2402 - len(gef_df)}.")

Count of None in 'country_iso3': 137
Count of None in 'country_iso3': 112


C:\Users\david\AppData\Local\Temp\ipykernel_11944\2055983589.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['country_iso3'] = filtered_df['title'].apply(extract_countries_to_iso3)


Total projects from the Adaptation Fund are 147. On the website, there are 149. Missing or not in dataset: 2.
Total projects from the GCF are 253. On the website, there are 253. Missing or not in dataset: 0.
Total projects from the GEF are 2402. On the website, there are 2402. Missing or not in dataset: 0.
